# Tool Use Anatomy

<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/llmu/tool_use_anatomy.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

*Read the accompanying [article here](https://cohere.ai/blog/tool-use-anatomy/).*

In this notebook, we’ll dissect the key components of a tool use system and what a tool use workflow looks like. And we’ll do that with a concrete code example.

We’ll look at a use case of a RAG assistant that can query the sales database of an e-commerce company.

First, let’s install the Cohere Python SDK and set up the Cohere client.

In [ ]:
! pip install cohere -q

In [1]:
import cohere
co = cohere.Client("COHERE_API_KEY") # Get your API key: https://dashboard.cohere.com/api-keys

# Tool use setup

The pre-requisite, or Step 0, before we can run a tool use workflow, is to set up the tools. We can break this further into two steps:
- Creating the tool
- Defining the tool schema


# 1. Create tool

Let’s create a function to query a sales database called daily_sales_report and represent it as a tool. For simplicity, it contains a mock database containing just three data entries and the logic to return the data given a user query.

In this example, we are defining a Python function as the tool. But more broadly, the tool can be any function or service that can receive and send data. It could be an email service, an SQL database, a vector database, a weather data service, a sports data service, a web search engine, or even another LLM, just to give a few examples.

In [12]:
def daily_sales_report(day: str) -> dict:
    """
    Function to retrieve the sales report for the given day
    """
    # Mock database containing daily sales reports
    sales_database = {
    '2023-09-28': {'total_sales_amount': 5000,'total_units_sold': 100},
    '2023-09-29': {'total_sales_amount': 10000,'total_units_sold': 250},
    '2023-09-30': {'total_sales_amount': 8000,'total_units_sold': 200}
    }
    
    report = sales_database.get(day, {})
    
    if report:
        return {
            'date': day,
            'summary': f"Total Sales Amount: {report['total_sales_amount']}, Total Units Sold: {report['total_units_sold']}"
        }
    else:
        return {'date': day, 'summary': 'No sales data available for this day.'}
    

functions_map = {
    "daily_sales_report": daily_sales_report
}

## 2. Define tool schema

Next, we define the tool schema for the sales database tool. This schema is what will be passed to the Cohere API when running a tool use workflow.

This schema informs the LLM about what the tool does, and the LLM decides whether to use a particular tool based on it. Therefore, the more descriptive and specific the schema, the more likely the LLM will make the right tool call decisions.

In [14]:
tools = [
    {
        "name": "daily_sales_report",
        "description": "Connects to a database to retrieve overall sales volumes and sales information for a given day.",
        "parameter_definitions": {
            "day": {
                "description": "Retrieves sales data for this day, formatted as YYYY-MM-DD.",
                "type": "str",
                "required": True
            }
        }
    }
]

## 3. Define the preamble (optional)

An optional step is to add a custom preamble, which is the LLM’s system message.

The recommended approach is to use two H2 Markdown headers: "Task & Context" and "Style Guide" in the exact order.

It’s a completely optional step, though it’s likely needed if we want to create a robust and reliable application. Also note that the preamble is not related to the tool setup that we covered earlier, rather it’s part of the instruction to the LLM.


In [15]:
preamble = """## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

# Tool use workflow

The tool is now ready to use. We can think of a tool use system as consisting of four components:
- The user
- The application
- The LLM
- The tools

At its most basic, these four components interact in a workflow through four steps:
- Step 1: Get user message. The LLM gets the user message (via the application).
- Step 2: Generate tool calls. The LLM makes a decision on the tools to call (if any) and generates the tool calls.
- Step 3: Get tool results. The tools are executed by the application and the results are sent to the LLM.
- Step 4: Generate response and citations. The LLM generates the response and citations to back to the user.

Let’s walk through these four steps using a code example. In this first example, we’ll use the simplest possible scenario where:
- There is only one tool (the sales database)
- Tool calling happens only once (and only one tool is called)
- There is only one turn in the conversation (no conversation memory preserved)


## 1. Get user message

The LLM gets the user message (via the application).

In [16]:
message = "Can you provide a sales summary for 29th September 2023?"

## 2. Generate tool calls

The LLM makes a decision on the tools to call (if any) and generates the tool calls.

In [26]:
model = "command-r-plus"

# Initial response to the user message
response = co.chat(
    message=message,
    model=model,
    preamble=preamble,
    tools=tools,
    force_single_step=True
)

tool_calls = response.tool_calls
    
print("Tool calls:\n")
for i, t in enumerate(tool_calls):
    print(f"#{i+1}\nTool: {t.name}\nParameters: {t.parameters}\n")

Tool calls:

#1
Tool: daily_sales_report
Parameters: {'day': '2023-09-29'}



## 3. Get tool results

The tools are executed by the application and the results are sent to the LLM.

In [27]:
tool_results = []
for tc in tool_calls:
    tool_output = functions_map[tc.name](**tc.parameters)
    tool_call = {"name": tc.name, "parameters": tc.parameters}
    tool_results.append({"call": tool_call, "outputs": [tool_output]})
    
print("Tool results:\n")
for i, t in enumerate(tool_results):
    print(f"#{i+1}\nTool call: {t['call']}\nOutputs: {t['outputs']}\n")

Tool results:

#1
Tool call: {'name': 'daily_sales_report', 'parameters': {'day': '2023-09-29'}}
Outputs: [{'date': '2023-09-29', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250'}]



## 4. Generate response and citations

The LLM generates the response and citations to back to the user.

In [28]:
# Generate response
response = co.chat(
    message="",
    model=model,
    preamble=preamble,
    tools=tools,
    tool_results=tool_results,
    chat_history=response.chat_history
)
    
print("Final response:")
print(response.text)
print("="*50)

if response.citations:
    print("\nCitations:")
    for citation in response.citations:
        print(citation)
    print("\nCited Documents:")
    for document in response.documents:
        print(document)

Final response:
On 29 September 2023, we sold 250 units, totalling 10,000 in sales.

Citations:
start=30 end=39 text='250 units' document_ids=['daily_sales_report:0:2:0']
start=51 end=66 text='10,000 in sales' document_ids=['daily_sales_report:0:2:0']

Cited Documents:
{'date': '2023-09-29', 'id': 'daily_sales_report:0:2:0', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250', 'tool_name': 'daily_sales_report'}


In [29]:
# Print chat history
for turn in response.chat_history:
    print(turn,"\n")

Chat history:

message='Can you provide a sales summary for 29th September 2023?' tool_calls=None role='USER'

message=None tool_calls=[ToolCall(name='daily_sales_report', parameters={'day': '2023-09-29'})] role='CHATBOT'

tool_results=[ToolResult(call=ToolCall(name='daily_sales_report', parameters={'day': '2023-09-29'}), outputs=[{'date': '2023-09-29', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250'}])] role='TOOL'

message='On 29 September 2023, we sold 250 units, totalling 10,000 in sales.' tool_calls=None role='CHATBOT'


# Streaming option

We can stream the response from the Chat endpoint for each generated token instead of having to wait for the full response. To enable streaming, we need to change the endpoint call from `co.chat` to `co.chat_stream`. 

In [21]:
# Generate response
response = co.chat_stream(
    message="",
    model=model,
    preamble=preamble,
    tools=tools,
    tool_results=tool_results,
    chat_history=response.chat_history
)
    
print("Final response:\n")
chatbot_response = ""

for event in response:
    if event.event_type == "text-generation":
        print(event.text, end="")
        chatbot_response += event.text
    if event.event_type == "stream-end":
        if event.response.citations:
            print("\n\nCitations:")
            for citation in event.response.citations:
                print(citation)
        if event.response.documents:
            print("\nCited Documents")
            for document in event.response.documents:
                print(document)

Final response:

On 29 September 2023, total sales amounted to 10,000, with 250 units sold.

Citations:
start=22 end=52 text='total sales amounted to 10,000' document_ids=['daily_sales_report:0:2:0', 'daily_sales_report:0:4:0']
start=59 end=73 text='250 units sold' document_ids=['daily_sales_report:0:2:0', 'daily_sales_report:0:4:0']

Cited Documents
{'date': '2023-09-29', 'id': 'daily_sales_report:0:2:0', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250', 'tool_name': 'daily_sales_report'}
{'date': '2023-09-29', 'id': 'daily_sales_report:0:4:0', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250', 'tool_name': 'daily_sales_report'}
